In [11]:
import os
import re
from datetime import datetime, timedelta
from collections import defaultdict
from astroquery.simbad import Simbad
import numpy as np
from astropy.io import fits

# Список объектов из журнала наблюдений
raw_objects = """
Iot Psc
Gam Cas
Phi And
BS 1040
Pleione
BS 1389
BS 1728
BS 1752
Alp UMi
QY Gem
FS CMa
HD 50138
3 Pup
HD 52961
HD 74721
"""

# Преобразование списка объектов
objects = [obj.strip().replace(" ", "") for obj in raw_objects.strip().split('\n')]

# Путь к директории для сохранения файлов
directory_path = '/Users/nadezhda/Documents/TCO_spectra/20240121'

# Вспомогательные функции
def extract_date_from_filename(directory_path):
    for filename in os.listdir(directory_path):
        match = re.match(r"(\d{8})-\d{6}-.*\.fit", filename)
        if match:
            return match.group(1)
    return None

def get_object_name(filename):
    parts = filename.split('-')
    return parts[2] if len(parts) > 2 else filename

# Получение даты наблюдений из названий файлов
observation_date = extract_date_from_filename(directory_path)

if observation_date is None:
    raise ValueError("Не удалось извлечь дату наблюдений из названий файлов.")

# Уменьшение даты на один день
observation_date = (datetime.strptime(observation_date, '%Y%m%d') - timedelta(days=1)).strftime('%Y%m%d')

# Создание и запись файла objname
objname_path = os.path.join(directory_path, "objname")
with open(objname_path, 'w') as objname_file:
    for obj in objects:
        objname_file.write(f"hedit\t*{obj}*\tOBJNAME\t{obj}\n")
print("objname")

# Создание и запись файла objfile
objfile_path = os.path.join(directory_path, "objfile")
with open(objfile_path, 'w') as objfile:
    for obj in objects:
        objfile.write(f"ls\t*{obj}*\t>\t{obj.lower()}\n")
print("objfile")

# Создание строки для команды ls
ls_command = "ls " + " ".join([f"*{obj}*" for obj in objects]) + " > obj"

# Содержимое файла all3.cl
all3_cl_content = f"""
string *list2
string s4,s5

ls *BIAS* > zero

ls *THAR* > comp

ls *FLAT* > flat

{ls_command}

hedit ("@zero",
"IMAGETYP", "zero", add=yes, delete=no, verify=no, show=yes, update=yes)

hedit ("@comp",
"IMAGETYP", "comp", add=yes, delete=no, verify=no, show=yes, update=yes)

hedit ("@flat",
"IMAGETYP", "flat", add=yes, delete=no, verify=no, show=yes, update=yes)

hedit ("@obj",
"IMAGETYP", "object", add=yes, delete=no, verify=no, show=yes, update=yes)

hsel ("@obj",
"$I,EXPOSURE", "yes", >> "imglist")

list="imglist" 
while (fscan (list, s1, x, y) != EOF) 
{{
hedit (s1,
"EXPTIME", x, add=yes, delete=no, verify=no, show=yes, update=yes)
}}
del imglist

hedit ("@comp",
"DISPAXIS", "1", add=yes, delete=no, verify=no, show=yes, update=yes)

hedit ("@flat",
"DISPAXIS", "1", add=yes, delete=no, verify=no, show=yes, update=yes)

hedit ("@obj",
"DISPAXIS", "1", add=yes, delete=no, verify=no, show=yes, update=yes)

hsel ("@obj",
"$I,OBJNAME", "yes", >> "imglist")

list="imglist" 
while (fscan (list, s1, s2) != EOF) 
{{
list2="coord_tco.txt"
while (fscan (list2, s3, s4, s5) != EOF) 
{{
if (s2==s3)
{{
print (s1," find")
hedit (s1,
"RA", s4, add=yes, delete=no, verify=no, show=no, update=yes)
hedit (s1,
"DEC", s5, add=yes, delete=no, verify=no, show=no, update=yes)
hedit (s1,
"EPOCH", "2000", add=yes, delete=no, verify=no, show=no, update=yes)
hedit (s1,
"OBSERVAT", "tco", add=yes, delete=no, verify=no, show=no, update=yes)
}}
else
x=1
}}
}}
del imglist

print (" ")
print ("Check New Keywords")
print (" ")

hsel ("@obj",
"$I,RA,DEC,EPOCH,IMAGETYP,DISPAXIS,OBSERVAT", "yes")
"""

# Путь к директории для сохранения файла all3.cl
all3_cl_path = os.path.join(directory_path, "all3.cl")

# Запись содержимого в файл all3.cl
with open(all3_cl_path, 'w') as all3_cl_file:
    all3_cl_file.write(all3_cl_content)

print("all3.cl")

# Содержимое файла zerocomb_ccdproc_tco.cl
zerocomb_ccdproc_tco_cl_content = f"""
zerocombine ("@zero", output="zero_{observation_date}", combine="average", reject="minmax", ccdtype="zero",
process=no, delete=no, clobber=no, scale="none", statsec="", nlow=0, nhigh=1,
nkeep=1, mclip=yes, lsigma=3., hsigma=3., rdnoise="5.", gain="0.26",
snoise="0.", pclip=-0.5, blank=0.)

ccdproc ("@comp",
output="b@comp", ccdtype="comp", max_cache=34, noproc=no, fixpix=yes,
overscan=no, trim=no, zerocor=yes, darkcor=no, flatcor=no, illumcor=no,
fringecor=no, readcor=no, scancor=no, readaxis="line", fixfile="badpix_atik_20231221",
biassec="", trimsec="", zero="zero_{observation_date}", dark="", flat="", illum="",
fringe="", minreplace=1., scantype="shortscan", nscan=1, interactive=no,
function="legendre", order=1, sample="*", naverage=1, niterate=1,
low_reject=3., high_reject=3., grow=0.)

ccdproc ("@obj",
output="b@obj", ccdtype="object", max_cache=34, noproc=no, fixpix=yes,
overscan=no, trim=no, zerocor=yes, darkcor=no, flatcor=no, illumcor=no,
fringecor=no, readcor=no, scancor=no, readaxis="line", fixfile="badpix_atik_20231221",
biassec="", trimsec="", zero="zero_{observation_date}", dark="", flat="", illum="",
fringe="", minreplace=1., scantype="shortscan", nscan=1, interactive=no,
function="legendre", order=1, sample="*", naverage=1, niterate=1,
low_reject=3., high_reject=3., grow=0.)

del comp

ls b*THAR* > comp
"""

# Путь к директории для сохранения файла zerocomb_ccdproc_tco.cl
zerocomb_ccdproc_tco_cl_path = os.path.join(directory_path, "zerocomb_ccdproc_tco.cl")

# Запись содержимого в файл zerocomb_ccdproc_tco.cl
with open(zerocomb_ccdproc_tco_cl_path, 'w') as zerocomb_ccdproc_tco_cl_file:
    zerocomb_ccdproc_tco_cl_file.write(zerocomb_ccdproc_tco_cl_content)

print("zerocomb_ccdproc_tco.cl")

# Создание и запись файла apall_commands
apall_commands_path = os.path.join(directory_path, "apall_tco.cl")

def get_brightest_star(objects):
    custom_simbad = Simbad()
    custom_simbad.add_votable_fields("flux(V)")
    brightest_star = None
    max_flux = float('inf')
    for obj in objects:
        result = custom_simbad.query_object(obj)
        if result is not None and result['FLUX_V'][0] < max_flux:
            max_flux = result['FLUX_V'][0]
            brightest_star = obj
    return brightest_star

brightest_star = get_brightest_star(objects)

if brightest_star is None:
    raise ValueError("Не удалось определить самую яркую звезду.")

objects.remove(brightest_star)
objects.insert(0, brightest_star)

with open(apall_commands_path, 'w') as apall_commands:
    for i, obj in enumerate(objects):
        suffix = "1" if i == 0 else ""
        apall_commands.write(f"apall (\"@{obj.lower()}\",0, output=\"a@{obj.lower()}{suffix}\", apertures=\"\", format=\"echelle\",references=\"last\", profiles=\"\", interactive=no,find=no, recenter=yes,resize=no, edit=no, trace=yes, fittrace=no,extract=yes, extras=no, review=no, line=INDEF, nsum=10, lower=-5., upper=5.,apidtable=\"\", b_function=\"chebyshev\", b_order=2, b_sample=\"-10:-6,6:10\",b_naverage=-3, b_niterate=0, b_low_reject=3., b_high_rejec=3., b_grow=0.,width=5., radius=10., threshold=0., minsep=5., maxsep=100000.,order=\"increasing\", aprecenter=\"\", npeaks=INDEF, shift=no, llimit=INDEF,ulimit=INDEF, ylevel=0.1, peak=yes, bkg=yes, r_grow=0., avglimits=no,t_nsum=10, t_step=3, t_nlost=3, t_function=\"legendre\", t_order=4,t_sample=\"*\", t_naverage=1, t_niterate=10, t_low_reject=3., t_high_rejec=3.,t_grow=0., background=\"median\", skybox=1, weights=\"none\", pfit=\"fit1d\",clean=no, saturation=INDEF, readnoise=\"5.\", gain=\"0.26\", lsigma=4., usigma=4.,nsubaps=1)\n")

print("apall_tco.cl")

# Обработка передержанной линии Ha
def get_ha_saturation_status(directory_path, x_coord=329, y_coord=873, x_range=50, y_range=50, threshold=65000):
    files_by_object = defaultdict(list)
    for filename in os.listdir(directory_path):
        if filename.endswith('.fit') and 'THAR' not in filename and 'BIAS' not in filename:
            object_name = get_object_name(filename)
            files_by_object[object_name].append(filename)

    ha_status = {}
    for object_name, files in files_by_object.items():
        has_saturated = False
        saturated_exposures = []
        normal_exposures = []
        for filename in files:
            try:
                filepath = os.path.join(directory_path, filename)
                hdu = fits.open(filepath)
                data = hdu[0].data

                if data is None:
                    continue

                x_start = max(x_coord - x_range // 2, 0)
                x_end = min(x_coord + x_range // 2, data.shape[1])
                y_start = max(y_coord - y_range // 2, 0)
                y_end = min(y_coord + y_range // 2, data.shape[0])

                region = data[y_start:y_end, x_start:x_end]
                max_value = np.max(region)

                match = re.match(r'^\d{8}-\d{6}-.*-(\d+)s-\d+.fit$', filename)
                if match:
                    exposure = int(match.group(1))
                    if max_value > threshold:
                        has_saturated = True
                        saturated_exposures.append(exposure)
                    else:
                        normal_exposures.append(exposure)

            except Exception as e:
                print(f"Ошибка при обработке файла {filename}: {e}")

        ha_status[object_name] = {
            'has_saturated': has_saturated,
            'saturated_exposures': saturated_exposures,
            'normal_exposures': normal_exposures
        }

    return ha_status

ha_saturation_status = get_ha_saturation_status(directory_path)

# Создание и запись файла scombine.cl
scombine_path = os.path.join(directory_path, "scombine.cl")
with open(scombine_path, 'w') as scombine_file:
    for obj in objects:
        status = ha_saturation_status.get(obj, {'has_saturated': False})
        if status['has_saturated']:
            # Записываем все экспозиции для lo
            scombine_file.write(f"scombine\tab*{obj}*\t{obj}_{observation_date}lo_nf\n")
            # Записываем нормальные экспозиции для sh
            sh_exposures = ",".join([f"ab*{obj}*{exp}s*" for exp in status['normal_exposures']])
            if sh_exposures:
                scombine_file.write(f"scombine\t{sh_exposures}\t{obj}_{observation_date}sh_nf\n")
        else:
            scombine_file.write(f"scombine\tab*{obj}*\t{obj}_{observation_date}nf\n")

print("scombine.cl")

# Создание и запись файлов compX и imcomb.cl
def create_comp_and_imcomb_files(directory_path):
    comp_files = {}
    imcomb_lines = []
    apall_lines = []
    thar_files = sorted([f for f in os.listdir(directory_path) if "THAR" in f])
    group_num = 1
    for i in range(0, len(thar_files), 5):
        comp_group = thar_files[i:i + 5]
        if not comp_group:
            break
        comp_filename = f"comp{group_num}"
        comp_files[comp_filename] = comp_group
        imcomb_lines.append(f"imcombine @{comp_filename} b{observation_date}-comp{group_num}-15s-1")
        apall_lines.append(f"""apall ("b{observation_date}-comp{group_num}-15s-1",
0, output="eb{observation_date}-comp{group_num}-15s-1", apertures="", format="echelle",
references="last", profiles="", interactive=no,
find=no, recenter=no, resize=no, edit=no, trace=yes, fittrace=no,
extract=yes, extras=no, review=no, line=INDEF, nsum=10, lower=-5., upper=5.,
apidtable="", b_function="chebyshev", b_order=2, b_sample="-10:-6,6:10",
b_naverage=-3, b_niterate=0, b_low_reject=3., b_high_rejec=3., b_grow=0.,
width=5., radius=10., threshold=0., minsep=5., maxsep=100000.,
order="increasing", aprecenter="", npeaks=INDEF, shift=no, llimit=INDEF,
ulimit=INDEF, ylevel=0.1, peak=yes, bkg=yes, r_grow=0., avglimits=no,
t_nsum=10, t_step=3, t_nlost=3, t_function="legendre", t_order=4,
t_sample="*", t_naverage=1, t_niterate=10, t_low_reject=3., t_high_rejec=3.,
t_grow=0., background="none", skybox=1, weights="none", pfit="fit1d",
clean=no, saturation=INDEF, readnoise="5.", gain="0.26", lsigma=4., usigma=4.,
nsubaps=1)\n""")
        group_num += 1

    for comp_filename, files in comp_files.items():
        with open(os.path.join(directory_path, comp_filename), 'w') as comp_file:
            for file in files:
                comp_file.write(f"{file}\n")
        print(f"{comp_filename}")

    imcomb_path = os.path.join(directory_path, "imcomb.cl")
    with open(imcomb_path, 'w') as imcomb_file:
        for line in imcomb_lines:
            imcomb_file.write(f"{line}\n")
    print(f"imcomb.cl")

    apall_comp_path = os.path.join(directory_path, "apall_comp.cl")
    with open(apall_comp_path, 'w') as apall_comp_file:
        for line in apall_lines:
            apall_comp_file.write(f"{line}\n")
    print(f"apall_comp.cl")

create_comp_and_imcomb_files(directory_path)

# Создание и запись файла ut.cl
def extract_time(file_name):
    match = re.match(r'^\d{8}-(\d{6})-.*-(\d+)s-\d+.fit$', file_name)
    if match:
        time_str = match.group(1)
        start_time = datetime.strptime(time_str, '%H%M%S')
        return start_time
    return None

def create_ut_file(directory_path, observation_date):
    objects = defaultdict(list)
    normal_objects = defaultdict(list)

    # Собрать времена для каждого файла
    for file in os.listdir(directory_path):
        if file.endswith('.fit'):
            match = re.match(r'^\d{8}-\d{6}-(.*)-\d+s-\d+.fit$', file)
            if match:
                object_name = match.group(1)
                if object_name != "THAR":
                    start_time = extract_time(file)
                    if start_time:
                        objects[object_name].append(start_time)
                        status = ha_saturation_status.get(object_name, {'has_saturated': False})
                        if not status['has_saturated']:
                            normal_objects[object_name].append(start_time)
                        elif object_name in ha_saturation_status and file in ha_saturation_status[object_name]['normal_exposures']:
                            normal_objects[object_name].append(start_time)

    # Вычислить среднее время для каждого объекта
    object_average_times = {}
    for object_name, times in objects.items():
        total_time = sum((t - datetime(1970, 1, 1)).total_seconds() for t in times)
        average_time_seconds = total_time / len(times)
        average_time = datetime(1970, 1, 1) + timedelta(seconds=average_time_seconds)
        object_average_times[object_name] = average_time

    # Вычислить среднее время для нормальных экспозиций
    normal_average_times = {}
    for object_name, times in normal_objects.items():
        total_time = sum((t - datetime(1970, 1, 1)).total_seconds() for t in times)
        average_time_seconds = total_time / len(times)
        average_time = datetime(1970, 1, 1) + timedelta(seconds=average_time_seconds)
        normal_average_times[object_name] = average_time

    sorted_objects = sorted(object_average_times.items(), key=lambda x: x[1])

    # Запись данных в ut.cl
    ut_path = os.path.join(directory_path, "ut.cl")
    with open(ut_path, 'w') as ut_file:
        for object_name, avg_time in sorted_objects:
            avg_time_str = avg_time.strftime('%H:%M')
            status = ha_saturation_status.get(object_name, {'has_saturated': False})
            if status['has_saturated']:
                ut_file.write(f"hedit\t{object_name}_{observation_date}lo_nf\tUT\t{avg_time_str}\n")
                if object_name in normal_average_times:
                    normal_avg_time_str = normal_average_times[object_name].strftime('%H:%M')
                    ut_file.write(f"hedit\t{object_name}_{observation_date}sh_nf\tUT\t{normal_avg_time_str}\n")
            else:
                ut_file.write(f"hedit\t{object_name}_{observation_date}nf\tUT\t{avg_time_str}\n")
    print("ut.cl")

create_ut_file(directory_path, observation_date)

# Создание и запись файла thar.cl
def create_thar_file(directory_path, observation_date):
    ut_path = os.path.join(directory_path, "ut.cl")
    
    # Extract object names and average times from ut.cl
    object_avg_times = []
    with open(ut_path, 'r') as ut_file:
        for line in ut_file:
            parts = line.strip().split()
            if len(parts) == 4:
                object_name = parts[1].split('_')[0]
                object_avg_times.append(object_name)
    
    # Track which objects have already been processed for lo and sh
    processed_objects = set()

    # Create thar.cl file
    thar_path = os.path.join(directory_path, "thar.cl")
    with open(thar_path, 'w') as thar_file:
        for i, object_name in enumerate(object_avg_times):
            comp_group = (i // 5) + 1
            status = ha_saturation_status.get(object_name, {'has_saturated': False})
            if status['has_saturated']:
                if f"{object_name}_lo" not in processed_objects:
                    thar_file.write(f"hedit\t{object_name}_{observation_date}lo_nf\tREFSPEC1\teb{observation_date}-comp{comp_group}-15s-1\n")
                    processed_objects.add(f"{object_name}_lo")
                if f"{object_name}_sh" not in processed_objects:
                    thar_file.write(f"hedit\t{object_name}_{observation_date}sh_nf\tREFSPEC1\teb{observation_date}-comp{comp_group}-15s-1\n")
                    processed_objects.add(f"{object_name}_sh")
            else:
                if f"{object_name}_nf" not in processed_objects:
                    thar_file.write(f"hedit\t{object_name}_{observation_date}nf\tREFSPEC1\teb{observation_date}-comp{comp_group}-15s-1\n")
                    processed_objects.add(f"{object_name}_nf")
    print("thar.cl")

create_thar_file(directory_path, observation_date)

# Создание и запись файлов obj.nowav и obj.wav
def create_obj_files(directory_path, observation_date, objects):
    nowav_path = os.path.join(directory_path, "obj_nowav")
    wav_path = os.path.join(directory_path, "obj_wav")
    
    with open(nowav_path, 'w') as nowav_file, open(wav_path, 'w') as wav_file:
        for obj in objects:
            status = ha_saturation_status.get(obj, {'has_saturated': False})
            if status['has_saturated']:
                nowav_file.write(f"{obj}_{observation_date}lo_nf.fits\n")
                nowav_file.write(f"{obj}_{observation_date}sh_nf.fits\n")
                wav_file.write(f"{obj}_{observation_date}lo_nfw.fits\n")
                wav_file.write(f"{obj}_{observation_date}sh_nfw.fits\n")
            else:
                nowav_file.write(f"{obj}_{observation_date}nf.fits\n")
                wav_file.write(f"{obj}_{observation_date}nfw.fits\n")
    
    print("obj.nowav")
    print("obj.wav")

create_obj_files(directory_path, observation_date, objects)

print("Обработка завершена.")

objname
objfile
all3.cl
zerocomb_ccdproc_tco.cl
apall_tco.cl


/Users/nadezhda/anaconda3/lib/python3.11/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'IotPsc': No known catalog could be found
  warnings.warn("Warning: The script line number %i raised "
/Users/nadezhda/anaconda3/lib/python3.11/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'GamCas' this identifier has an incorrect format for catalog: 	G : Giclas
  warnings.warn("Warning: The script line number %i raised "
/Users/nadezhda/anaconda3/lib/python3.11/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'PhiAnd': No known catalog could be found
  warnings.warn("Warning: The script line number %i raised "
/Users/nadezhda/anaconda3/lib/python3.

scombine.cl
comp1
comp2
comp3
comp4
imcomb.cl
apall_comp.cl
ut.cl
thar.cl
obj.nowav
obj.wav
Обработка завершена.
